## Example of Pyspark ML

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Missing").getOrCreate()

In [4]:
### Read the dataset
training = spark.read.csv('Data4.csv', header=True, inferSchema=True)

In [5]:
training.show()

+---------+---+-----------+------+
|    Name |Age|Experience |Salary|
+---------+---+-----------+------+
|    Krish| 31|         19| 30000|
|Sudhanshu| 30|          8| 25000|
|    Sunny| 29|          4| 20000|
|     Paul| 24|          3| 20000|
|   Harsha| 21|          1| 15000|
|  Shubham| 23|          2| 18000|
+---------+---+-----------+------+



In [6]:
training.printSchema()

root
 |-- Name : string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience : integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
training.columns

['Name ', 'Age', 'Experience ', 'Salary']

In [14]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["Age", "Experience "], outputCol="Independent Features")

In [15]:
output = featureassembler.transform(training)

In [16]:
output.show()

+---------+---+-----------+------+------------------+
|    Name |Age|Experience |Salary|Inpenrent Features|
+---------+---+-----------+------+------------------+
|    Krish| 31|         19| 30000|       [31.0,19.0]|
|Sudhanshu| 30|          8| 25000|        [30.0,8.0]|
|    Sunny| 29|          4| 20000|        [29.0,4.0]|
|     Paul| 24|          3| 20000|        [24.0,3.0]|
|   Harsha| 21|          1| 15000|        [21.0,1.0]|
|  Shubham| 23|          2| 18000|        [23.0,2.0]|
+---------+---+-----------+------+------------------+

